In [1]:
import sys
sys.path.append("/data1/andrew/meng/mixehr/meng/VAE-EHR/src")
import os
print(sys.version)

from vae import VAE, VAETrainer
from config_reader import Config
from icd_analysis_helper import ICDAnalysisHelper
from visualizer_helper import Visualizer
#from patient_cluster import PatientCluster

from typing import List, Tuple
import numpy as np 
import pandas as pd
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

import hdbscan
from wordcloud import WordCloud
from collections import Counter

import pickle

3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]


In [2]:
config = Config(config_file_path='./config.ini', date='200117')
config.__dict__

{'patient_icd_path': '/data1/andrew/meng/mixehr/data/Mimic/andrew_outputs/PATIENT_ICD_BINARY.csv',
 'icd9codes_path': '/data1/andrew/meng/mixehr/data/Mimic/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz',
 'experiment_name': '200117_patient_clusters_25_latent_dim_beta_vae_with_larger_decoder',
 'encoder_dim': [(250, 500), (500, 250), (250, 100)],
 'latent_dim': 25,
 'decoder_dim': [(250, 500), (500, 250), (250, 100)],
 'use_relu_encoder': True,
 'use_relu_decoder': True,
 'kld_beta': 2.0}

In [3]:
patient_icd_df=pd.read_csv(config.patient_icd_path, sep=' ')
patient_icd_data = patient_icd_df.drop('SUBJECT_ID', axis=1)
data = torch.tensor(patient_icd_data.values).float()
print(data.shape)

torch.Size([46520, 6984])


In [4]:
icd9codes = pd.read_csv(config.icd9codes_path)
icd_analyzer = ICDAnalysisHelper(icd9codes_df = icd9codes, patient_icd_df = patient_icd_df)

visualizer = Visualizer()

In [5]:
print("Feature_dim: {}".format(data.shape[1]))
model = VAE(
    feature_dim = data.shape[1], 
    encoder_dim = config.encoder_dim,
    latent_dim = config.latent_dim,
    decoder_dim = config.decoder_dim,
    use_relu_encoder= config.use_relu_encoder,
    use_relu_decoder= config.use_relu_decoder
)

optimizer = optim.Adam(model.parameters(), lr=0.001)
print(type(optimizer))

experiment_name=config.experiment_name
epoch = 55
model.load_state_dict(torch.load("./VAE_exp_{}_epoch_{}.pkl".format(experiment_name, epoch)))

trainer = VAETrainer(
    model=model, 
    optimizer=optimizer,
    experiment_name=experiment_name,
    kld_beta=config.kld_beta
)

Feature_dim: 6984
<class 'torch.optim.adam.Adam'>


In [6]:
latent, means, var = trainer.encode_data(data)
latent_umap = visualizer.umap_embedding(latent.cpu().detach().numpy())

/data1/andrew/meng/mixehr/meng/VAE-EHR/src/visualizer_helper.py:18: UserWarning: Parameter should_downcast is deprecated, use convert_dtype in fit, fit_transform and transform  methods instead. 
  return UMAP(n_components=n_components).fit_transform(X)


In [7]:
latent_hdbscan = hdbscan.HDBSCAN(min_cluster_size=100, cluster_selection_epsilon=1.0)
latent_clustering = latent_hdbscan.fit_predict(latent.cpu().detach().numpy())

In [8]:
umap_hdbscan = hdbscan.HDBSCAN(min_cluster_size=100, cluster_selection_epsilon=1.0)
umap_clustering = umap_hdbscan.fit_predict(latent_umap)

In [9]:
unique_clusters = np.sort(np.unique(latent_clustering))
unique_umap_clusters = np.sort(np.unique(umap_clustering))
print("Latent Clusters:", unique_clusters)
print("UMAP Clusters:", unique_umap_clusters)
print("Clusters and number of members:", Counter(latent_clustering))

Latent Clusters: [-1  0  1]
UMAP Clusters: [0 1 2]
Clusters and number of members: Counter({-1: 44132, 1: 2136, 0: 252})


In [10]:
cluster_colors = umap_clustering
tag = "umap_feature_clusters"
visualizer.plot2d(
    X=latent_umap, 
    filename="Patient_Clusters_exp_{}_epoch_{}_{}".format(experiment_name, epoch, tag), 
    colors=cluster_colors*100, 
    alpha = 0.01)

In [11]:
clustering = umap_clustering
patient_icd_df_with_cluster_assignments = patient_icd_df.assign(CLUSTER=clustering)
patient_icd_df_with_cluster_assignments

SUBJECT_ID  0030  0031  0038  0039  0041  0048  0049  0051  00581  ...  \
0             109     0     0     0     0     0     0     0     0      0  ...   
1             112     0     0     0     0     0     0     0     0      0  ...   
2             113     0     0     0     0     0     0     0     0      0  ...   
3             114     0     0     0     0     0     0     0     0      0  ...   
4             115     0     0     0     0     0     0     0     0      0  ...   
5             116     0     0     0     0     0     0     0     0      0  ...   
6             117     0     0     0     0     0     0     0     0      0  ...   
7             118     0     0     0     0     0     0     0     0      0  ...   
8             119     0     0     0     0     0     0     0     0      0  ...   
9             120     0     0     0     0     0     0     0     0      0  ...   
10            122     0     0     0     0     0     0     0     0      0  ...   
11            123     0     0     0     0     0     0     0     0      0  ...   
12            124     0     0     0     0     0     0     0     0      0  ...   
13            125     0     0     0     0     0     0     0     0      0  ...   
14            126     0     0     0     0     0     0     0     0      0  ...   
15            127     0     0     0     0     0     0     0     0      0  ...   
16            128     0     0     0     0     0     0     0     0      0  ...   
17            129     0     0     0     0     0     0     0     0      0  ...   
18            130     0     0     0     0     0     0     0     0      0  ...   
19            131     0     0     0     0     0     0     0     0      0  ...   
20            132     0     0     0     0     0     0     0     0      0  ...   
21            133     0     0     0     0     0     0     0     0      0  ...   
22            134     0     0     0     0     0     0     0     0      0  ...   
23            135     0     0     0     0     0     0     0     0      0  ...   
24            136     0     0     0     0     0     0     0     0      0  ...   
25            137     0     0     0     0     0     0     0     0      0  ...   
26            138     0     0     0     0     0     0     0     0      0  ...   
27             64     0     0     0     0     0     0     0     0      0  ...   
28             65     0     0     0     0     0     0     0     0      0  ...   
29             66     0     0     0     0     0     0     0     0      0  ...   
...           ...   ...   ...   ...   ...   ...   ...   ...   ...    ...  ...   
46490       95787     0     0     0     0     0     0     0     0      0  ...   
46491       95793     0     0     0     0     0     0     0     0      0  ...   
46492       95798     0     0     0     0     0     0     0     0      0  ...   
46493       95803     0     0     0     0     0     0     0     0      0  ...   
46494       95806     0     0     0     0     0     0     0     0      0  ...   
46495       95810     0     0     0     0     0     0     0     0      0  ...   
46496       95811     0     0     0     0     0     0     0     0      0  ...   
46497       95816     0     0     0     0     0     0     0     0      0  ...   
46498       96004     0     0     0     0     0     0     0     0      0  ...   
46499       96006     0     0     0     0     0     0     0     0      0  ...   
46500       96008     0     0     0     0     0     0     0     0      0  ...   
46501       96010     0     0     0     0     0     0     0     0      0  ...   
46502       96015     0     0     0     0     0     0     0     0      0  ...   
46503       96016     0     0     0     0     0     0     0     0      0  ...   
46504       96017     0     0     0     0     0     0     0     0      0  ...   
46505       96023     0     0     0     0     0     0     0     0      0  ...   
46506       97143     0     0     0     0     0     0     0     0      0  ...   
46507       97144     0     0     0   

In [14]:
class PatientCluster:
    def __init__(self, 
                patient_icd_df_with_cluster_assignments, 
                cluster):
        self.cluster = cluster
        self.cluster_df = patient_icd_df_with_cluster_assignments[patient_icd_df_with_cluster_assignments.CLUSTER == cluster]
        
        self.disease_df = self.cluster_df.drop(['SUBJECT_ID', 'CLUSTER'], axis=1)
        self.disease_distribution = self.disease_df.sum(axis=0).sort_values(ascending=False)
        
    def get_cluster_ids(self):
        return self.cluster_df.SUBJECT_ID

    def plot_disease_distribution(self, top_k_diseases: int=10, use_short_titles: bool=True):
        
        disease_distribution = self.disease_distribution[:top_k_diseases].sort_values(ascending=True)
        plt.figure(figsize=(5, 10))
        plt.barh(np.arange(disease_distribution.size),
           disease_distribution.to_numpy(),
           align='center')
        
        if use_short_titles:
            title_keys = map(lambda icd9code: ICDAnalysisHelper.get_icd_short_titles([icd9code]), 
                                disease_distribution.keys() )
        else:
            title_keys = map(lambda icd9code: ICDAnalysisHelper.get_icd_long_titles([icd9code]), 
                                disease_distribution.keys() )
        plt.yticks(np.arange(disease_distribution.size), title_keys)
        plt.xlim(0, np.max(disease_distribution))
        plt.title("Disease Distribution for Cluster {}".format(self.cluster))
        plt.show()

    def generate_wordcloud(self,
                           use_short_titles:bool=False,
                           weight_by_disease_frequency:bool=True,
                           max_words_in_cloud=200,
                           verbose=True
                          ):
        ###Titles
        disease_freqs = self.disease_distribution.values
        
        if use_short_titles:
            titles = map(lambda icd9code: ICDAnalysisHelper.get_icd_short_titles([icd9code], verbose=verbose),self.disease_distribution.index )
        else:
            titles = map(lambda icd9code: ICDAnalysisHelper.get_icd_long_titles([icd9code], verbose=verbose),self.disease_distribution.index )
        
        ###Title Word Frequencies
        text_processor = WordCloud()
        all_word_freqs = Counter()
        for i, title in enumerate(titles):
            title_word_counts = text_processor.process_text(title)
            
            #Reset all word counts to 1. This is in case the long title has multiple occurences of the same word
            title_word_counts.update((word, 1) for word in title_word_counts.keys())
            
            #Weight word by number of occurences in cluster
            if weight_by_disease_frequency:
                title_word_counts.update((word, disease_freqs[i]*count) for word, count in title_word_counts.items())
            
            all_word_freqs.update(title_word_counts)
                
        cloud = WordCloud(max_words=max_words_in_cloud, 
                          background_color="white",
                          colormap="cool").generate_from_frequencies(all_word_freqs)
        
        plt.figure(figsize=(10, 10))
        plt.title("WordCloud for Patient Cluster {}".format(self.cluster))
        plt.imshow(cloud, interpolation='bilinear')
        plt.axis('off')
        plt.show()
        

In [16]:
unique_clusters = unique_umap_clusters
patient_clusters = []
for cluster in unique_clusters: 
    patient_clusters.append(PatientCluster(patient_icd_df_with_cluster_assignments, cluster))
for pc in patient_clusters:
    pc.plot_disease_distribution(use_short_titles=False)
    pc.generate_wordcloud(use_short_titles=False, verbose=False)

No long title found for ICD code ['7793']


In [ ]:
cluster_neg_1.generate_wordcloud(use_short_titles=False)

In [ ]:
latent_umap = visualizer.umap_embedding(latent.cpu().detach().numpy())
cluster_colors = latent_clustering


visualizer.plot2d(
    X=latent_umap, 
    filename="Patient_Clusters_exp_{}_epoch_{}{}".format(experiment_name, epoch, "_clusters_umap"), 
    colors=cluster_colors*100)